<center><font size="20">目标检测</font></center>

&emsp;在图像分类任务中，我们假设图像中只有一个主要物体对象，我们只关注如何识别其类别。 然而，很多时候图像里有多个我们感兴趣的目标，我们不仅想知道它们的类别，还想得到它们在图像中的具体位置。 在计算机视觉里，我们将这类任务称为目标检测（object detection）或物体检测。

&emsp;目标检测在多个领域中被广泛使用。 例如，在无人驾驶里，我们需要通过识别拍摄到的视频图像里的车辆、行人、道路和障碍的位置来规划行进线路。 机器人也常通过该任务来检测感兴趣的目标。安防领域则需要检测异常目标，如歹徒或者炸弹。


1. R-CNN
2. Fast R-CNN
3. Faster R-CNN
4. Mask R-CNN
5. YOLO

# R-CNN

&emsp;R-CNN 首先从输入图像中选取若干（例如2000个）提议区域（如锚框也是一种选取方法），并标注它们的类别和边界框（如偏移量）。 然后，用卷积神经网络对每个提议区域进行前向计算以抽取其特征。 接下来，我们用每个提议区域的特征来预测类别和边界框。

![R-CNN](https://zh-v2.d2l.ai/_images/r-cnn.svg)


&emsp;具体来说，R-CNN包括以下四个步骤： 

1.  对输入图像使用 *选择性搜索* 来选取多个高质量的提议区域 。这些提议区域通常是在多个尺度下选取的，并具有不同的形状和大小。每个提议区域都将被标注类别和真实边界框。

2.  选择一个预训练的卷积神经网络，并将其在输出层之前截断。将每个提议区域变形为网络需要的输入尺寸，并通过前向计算输出抽取的提议区域特征。 

3.  将每个提议区域的特征连同其标注的类别作为一个样本。训练多个支持向量机对目标分类，其中每个支持向量机用来判断样本是否属于某一个类别。

4.  将每个提议区域的特征连同其标注的边界框作为一个样本，训练线性回归模型来预测真实边界框。

&emsp;尽管 R-CNN 模型通过预训练的卷积神经网络有效地抽取了图像特征，但它的速度很慢。 想象一下，我们可能从一张图像中选出上千个提议区域，这需要上千次的卷积神经网络的前向计算来执行目标检测。 这种庞大的计算量使得 R-CNN 在现实世界中难以被广泛应用。

# Fast R-CNN

&emsp;R-CNN 的主要性能瓶颈在于，对每个提议区域，卷积神经网络的前向计算是独立的，而没有共享计算。
由于这些区域通常有重叠，独立的特征抽取会导致重复的计算。
 Fast R-CNN对 R-CNN 的主要改进之一，是仅在整张图象上执行卷积神经网络的前向计算。  

![Fast R-CNN模型](https://zh-v2.d2l.ai/_images/fast-rcnn.svg)

&emsp;Fast R-CNN的主要计算如下： 

1. 与 R-CNN 相比，Fast R-CNN 用来提取特征的卷积神经网络的输入是整个图像，而不是各个提议区域。此外，这个网络通常会参与训练。设输入为一张图像，将卷积神经网络的输出的形状记为 $1 \times c \times h_1  \times w_1$。

1. 假设选择性搜索生成了$n$个提议区域。这些形状各异的提议区域在卷积神经网络的输出上分别标出了形状各异的兴趣区域。然后，这些感兴趣的区域需要进一步抽取出形状相同的特征（比如指定高度$h_2$和宽度$w_2$），以便于连结后输出。为了实现这一目标，Fast R-CNN 引入了 *兴趣区域 (RoI) 池化* 层：将卷积神经网络的输出和提议区域作为输入，输出连结后的各个提议区域抽取的特征，形状为$n \times c \times h_2 \times w_2$。

1. 通过全连接层将输出形状变换为$n \times d$，其中超参数$d$取决于模型设计。

1. 预测$n$个提议区域中每个区域的类别和边界框。更具体地说，在预测类别和边界框时，将全连接层的输出分别转换为形状为 $n \times q$（$q$ 是类别的数量）的输出和形状为 $n \times 4$ 的输出。其中预测类别时使用 softmax 回归。

# Faster R-CNN

&emsp;为了较精确地检测目标结果，Fast R-CNN 模型通常需要在选择性搜索中生成大量的提议区域。
*Faster R-CNN* 提出将选择性搜索替换为 *区域提议网络*（region proposal network），从而减少提议区域的生成数量，并保证目标检测的精度。 

![Faster R-CNN 模型](https://zh-v2.d2l.ai/_images/faster-rcnn.svg)

&emsp;与Fast R-CNN 相比，Faster R-CNN 只将生成提议区域的方法从选择性搜索改为了区域提议网络，模型的其余部分保持不变。具体来说，区域提议网络的计算步骤如下： 

1. 使用填充为1的 $3\times 3$ 的卷积层变换卷积神经网络的输出，并将输出通道数记为 $c$。这样，卷积神经网络为图像抽取的特征图中的每个单元均得到一个长度为 $c$ 的新特征。

1. 以特征图的每个像素为中心，生成多个不同大小和宽高比的锚框并标注它们。

1. 使用锚框中心单元长度为 $c$ 的特征，分别预测该锚框的二元类别（含目标还是背景）和边界框。

1. 使用非极大值抑制，从预测类别为目标的预测边界框中移除相似的结果。最终输出的预测边界框即是兴趣区域汇聚层所需的提议区域。

# Mask R-CNN

&emsp;如果在训练集中还标注了每个目标在图像上的像素级位置，那么 *Mask R-CNN* 能够有效地利用这些详尽的标注信息进一步提升目标检测的精度。 

![Mask R-CNN 模型](https://zh-v2.d2l.ai/_images/mask-rcnn.svg)

&emsp;Mask R-CNN 是基于 Faster R-CNN 修改而来的。具体来说，Mask R-CNN 将兴趣区域汇聚层替换为了
*兴趣区域 (RoI) 对齐* 层，使用 *双线性插值*（bilinear interpolation）来保留特征图上的空间信息，从而更适于像素级预测。
兴趣区域对齐层的输出包含了所有与兴趣区域的形状相同的特征图。
它们不仅被用于预测每个兴趣区域的类别和边界框，还通过额外的全卷积网络预测目标的像素级位置。

# YOLO

&emsp;人类视觉系统快速且精准，只需瞄一眼（You Only Look Once，YOLO）即可识别图像中物品及其位置。

&emsp;传统目标检测系统采用deformable parts models (DPM)方法，通过滑动框方法提出目标区域，然后采用分类器来实现识别。近期的R-CNN类方法采用region proposal methods，首先生成潜在的bounding boxes，然后采用分类器识别这些bounding boxes区域。最后通过post-processing来去除重复bounding boxes来进行优化。这类方法流程复杂，存在速度慢和训练困难的问题。

&emsp;在YOLO中，将目标检测问题转换为直接从图像中提取bounding boxes和类别概率的单个回归问题，只需一眼（you only look once，YOLO）即可检测目标类别和位置。

![](https://pic2.zhimg.com/80/v2-2c4e8576b987236de47f91ad594bf36d_720w.png)

## YOLO与其他方法的比较

&emsp;YOLO将物体检测作为回归问题求解。基于一个单独的end-to-end网络，完成从原始图像的输入到物体位置和类别的输出。从网络设计上，YOLO与rcnn、fast rcnn及faster rcnn的区别如下：

&emsp;1. YOLO训练和检测均是在一个单独网络中进行。YOLO没有显示地求取region proposal的过程。而rcnn/fast rcnn 采用分离的模块（独立于网络之外的selective search方法）求取候选框（可能会包含物体的矩形区域），训练过程因此也是分成多个模块进行。Faster rcnn使用RPN（region proposal network）卷积网络替代rcnn/fast rcnn的selective search模块，将RPN集成到fast rcnn检测网络中，得到一个统一的检测网络。尽管RPN与fast rcnn共享卷积层，但是在模型训练过程中，需要反复训练RPN网络和fast rcnn网络（注意这两个网络核心卷积层是参数共享的）。

&emsp;2. YOLO将物体检测作为一个回归问题进行求解，输入图像经过一次inference，便能得到图像中所有物体的位置和其所属类别及相应的置信概率。而rcnn/fast rcnn/faster rcnn将检测结果分为两部分求解：物体类别（分类问题），物体位置即bounding box（回归问题）。

![YOLO](https://pic4.zhimg.com/80/v2-181f533e731b216a98b238af42ffbb2b_720w.png)